In [4]:
########################################################################################################################
# This script extracts the number of psychology and/or psychiatry visits of patients.
# Remark: Run this script only after P01_Subject_Inclusion.ipynb
########################################################################################################################

In [3]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from ast import literal_eval
from itertools import product
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [5]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [6]:
########################################################################################################################
# USER_SPECIFIC SETTING
# TARGET_DIR_PATH: Path of the directory storing the target datasets (created in P01_Subject_Inclusion.ipynb)
# ENC_PATH: Path of the encounter-level dataset (created in C07_Date_Adjustment.ipynb)
########################################################################################################################
TARGET_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'
ENC_PATH: str = '../00_Data/01_Cleaned_Data/Encounter_full_v2.parquet'

In [4]:
####################################################################################################################
# Load the cleaned encounter-level data
####################################################################################################################
df_enc: pd.DataFrame = pd.read_parquet(ENC_PATH)
df_enc.rename(columns={'EncounterKey': 'OutcomeEncKey'}, inplace=True)

In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds
########################################################################################################################
for conf_idx, (C, D) in enumerate(product(Cs, Ds), 1):
    log_head: str = f'[{conf_idx}. C={C}; D={D}] '
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the target dataset
    ####################################################################################################################
    y_path: str = os.path.join(TARGET_DIR_PATH, f'{C}_encounters_{D}_days_v1.csv')
    assert os.path.exists(y_path)
    df_y: pd.DataFrame = pd.read_csv(y_path, 
                                     usecols=['PatientDurableKey', 
                                              'Sex^0=Female', 
                                              'Sex^1=Male', 
                                              'OutcomeEncKey', 
                                              'OutcomeLabel'])

    ####################################################################################################################
    # Merge the two datasets
    ####################################################################################################################
    needed_cols: list[str] = ['PatientDurableKey', 'PsychiatryVisitCount', 'PsychologyVisitCount', 'OutcomeEncKey']
    df: pd.DataFrame = pd.merge(left=df_y, right=df_enc[needed_cols],
                                on=['PatientDurableKey', 'OutcomeEncKey'], how='left')
    target_col: str = 'OutcomeLabel'
    print(f'{log_head}Distribution of the target label:')
    print(f'{df[target_col].value_counts().to_dict()} ({df[target_col].mean()*100:.2f}%)')
    print('*'*120)

    ####################################################################################################################
    # Stratify df by sex and the target label
    ####################################################################################################################
    for target, sex in product([0, 1], ['Total', 'Female', 'Male']):
        if sex == 'Female':
            sex_cond = df['Sex^0=Female'] == 1
        elif sex == 'Male':
            sex_cond = df['Sex^1=Male'] == 1
        else:
            sex_cond = True 
        df_sub = df[(sex_cond) & (df[target_col] == target)]
        n_sub: int = df_sub.shape[0]
        log_head_sub: str = f'[C={C}, D={D}, Sex={sex}, Target={target}] '
        print(f'{log_head_sub}n={n_sub}')
        print('-'*120)

        ################################################################################################################
        # Compute the statistics in counts and percentages
        ################################################################################################################
        n_psychi_any: int = df_sub[df_sub['PsychiatryVisitCount'] > 0].shape[0] 
        n_psychi_none: int = df_sub[df_sub['PsychiatryVisitCount'] == 0].shape[0] 
        r_psychi_any: float = round(100* n_psychi_any / n_sub, 1)
        r_psychi_none: float = round(100* n_psychi_none / n_sub, 1)
        
        n_psycho_any: int = df_sub[df_sub['PsychologyVisitCount'] > 0].shape[0] 
        n_psycho_none: int = df_sub[df_sub['PsychologyVisitCount'] == 0].shape[0] 
        r_psycho_any: float = round(100* n_psycho_any / n_sub, 1)
        r_psycho_none: float = round(100* n_psycho_none / n_sub, 1)

        n_either_any: int = df_sub[(df_sub['PsychiatryVisitCount'] + df_sub['PsychologyVisitCount']) > 0].shape[0] 
        n_either_none: int = df_sub[(df_sub['PsychiatryVisitCount'] + df_sub['PsychologyVisitCount']) == 0].shape[0] 
        r_either_any: float = round(100* n_either_any / n_sub, 1)
        r_either_none: float = round(100* n_either_none / n_sub, 1)

        print(log_head_sub + f'#Psychiatry: [Y] {n_psychi_any} ({r_psychi_any}%); [N] {n_psychi_none} ({r_psychi_none}%)')
        print(log_head_sub + f'#Psychology: [Y] {n_psycho_any} ({r_psycho_any}%); [N] {n_psycho_none} ({r_psycho_none}%)')
        print(log_head_sub + f'#Either: [Y] {n_either_any} ({r_either_any}%); [N] {n_either_none} ({r_either_none}%)')
        print('-'*120)
        
    print('*'*120)
